# Создание Телеграм-ботов. Больше о pyTelegramBotAPI

Сегодня мы изучим еще пару фишек библиотеки pyTelegramBotAPI: научимся создавать кнопки к сообщениям, редактировать и удалять сообщения, вызывать всплывающие окна. Во второй половине лекции напишем бота-парсер новостного сайта.

---

Ссылки:
* Описание всех методов и классов Telegram Bot API: https://core.telegram.org/bots/api
* Краткая документация по pyTelegramBotAPI: https://pypi.org/project/pyTelegramBotAPI/
* Справочник по Telegram Bot API: https://tlgrm.ru/docs/bots/api
* Исходный код библиотеки pyTelegramBotAPI: https://github.com/eternnoir/pyTelegramBotAPI
* Гайд по созданию Телеграм-ботов №1: https://m.habr.com/ru/post/350648/
* Гайд по созданию Телеграм-ботов №2: https://mastergroosha.github.io/telegram-tutorial/

# Редактирование/удаление сообщений

Бот может редактировать уже отправленные сообщения, медиа, кнопки и т.д. Для этого используются методы, начинающиеся на `edit_message_`.  

Бот может удалять отправленные сообщения. Для этого используется метод `delete_message()`.  

Чтобы идентифицировать конкретное сообщение - нам понадобится связка `chat_id` + `message_id`.

In [ ]:
# Меняет отправленное сообщение
bot.edit_message_text(chat_id=message.chat.id, message_id=message.message_id, text='Новый текст')

# Меняет отправленную кнопку к сообщению (или создает ее)
markup = types.InlineKeyboardMarkup()  # Если не создавать кнопки в клавиатуру, то она исчезнет, если была
bot.edit_message_reply_markup(message.chat.id, message.message_id, reply_markup=markup)

# Меняет отправленное медиа
# Недостаточно просто написать media=<название файла>. Нужно задать медиа через types.InputMedia()
bot.edit_message_media(media=types.InputMedia(type='photo', media=image, caption=text, parse_mode='HTML'),
                           chat_id=message.chat.id, message_id=message.message_id, reply_markup=keyboard)

# Меняет подпись к медиа (по сути, то же самое, что и предыдущий метод, только оставляет прежний файл)
bot.edit_message_caption(caption="Подпись к медиа", chat_id=CHAT_ID, message_id=MESSAGE_ID)

# Редактирует сообщение с геопозицией
bot.edit_message_live_location(latitude=LATITUDE, longitude=LONGITUDE, chat_id=CHAT_ID, message_id=MESSAGE_ID)

# Удаляет сообщение
bot.delete_message(message.chat.id, message.message_id)


Кроме этого, бот может реагировать на то, что пользователь отредактировал свое сообщение. Для этого существует специальный хэндлер `edited_message_handler`. Принцип работы у него точно такой же, как и у обычного `message_handler`.  

Этот код отвечает на слово пользователя в формате `Сам <слово пользователя>`. Если пользователь отредактирует свое слово - бот поменяет его в своем ответе.

In [ ]:
@bot.message_handler(content_types=['text'])
def reply_to_message(message):
    bot.reply_to(message, "Сам {}".format(message.text))  # Отвечать на сообщения можно и так


@bot.edited_message_handler(content_types=['text'])
def edit_message(message):
    bot.edit_message_text(chat_id=message.chat.id,
                          text="Сам {}".format(message.text),
                          message_id=message.message_id + 1)  # Тут +1, потому что хотим поменять не сообщение пользователя,
                                                              # а следующее за ним, т.е. наше.

# Кнопки к сообщениям

К каждому сообщению бот может прикрепить кнопки, которые будут совершать некое действие.

In [ ]:
keyboard = types.InlineKeyboardMarkup()
url_button = types.InlineKeyboardButton(text="Перейти на Яндекс", url="https://ya.ru")
keyboard.add(url_button)
bot.send_message(message.chat.id, "Привет! Нажми на кнопку и перейди в поисковик.", reply_markup=keyboard)

Кнопки располагаются на инлайн-клавиатуре `InlineKeyboardMarkup`. Сами кнопки имеют тип `InlineKeyboardButton` и могут быть трех типов в зависимости от аргументов при создании.

1. **URL-кнопки** - отправляют пользователя по ссылке, которая в них записана. Такая кнопка используется в предыдущем примере.
2. **Callback-кнопки** - вызывают функцию в коде.
3. **Switch-кнопки** - отправляют сообщение с инлайновым запросом к боту выбранному пользователю. О них - в следующей лекции.

# Подробнее о типах кнопок

## Callback-кнопки

Callback-кнопка задается так:  
`callback_button = types.InlineKeyboardButton(text="Нажми меня", callback_data='test')`  
Здесь `callback_data` - строка с информацией о кнопке.  

Нажатие на этот тип кнопки приведет к отправке боту объекта `CallbackQuery` с полями `Message` (сообщение, к которому относилась кнопка) и `data` (строка, которую мы изначально заложили в кнопку).  

Отлавливать `CallbackQuery` нужно специальным хэндлером:

In [ ]:
@bot.callback_query_handler(func=lambda call: call.data == 'test')  # Вызовется, если поймали нужную кнопку
def callback_processing(call):
    # Здесь можно что-то сделать с сообщением. Например, заменить в нем текст.
    bot.edit_message_text(chat_id=call.message.chat.id, message_id=call.message.message_id, text="Пыщь")

## Всплывающие окна

В ответ на действие callback-кнопок Телеграм может показывать либо всплывающее уведомление, либо окно.  

Для этого в функции, которая обрабатывает callback, надо написать:

In [ ]:
# Если show_alert == False, то будет показано всплывающее уведомление. Иначе - всплывающее окно.
bot.answer_callback_query(callback_query_id=call.id, show_alert=False, text="Пыщь!")

# Бот-парсер новостного сайта

In [ ]:
# Жду код от Артема